<a href="https://colab.research.google.com/github/eisbetterthanpi/pytorch/blob/main/JEPA_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### setup

In [ ]:
# https://openreview.net/pdf?id=BZ5a1r-kVsf
# import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import collections
device = "cuda" if torch.cuda.is_available() else "cpu"
!pip install optuna
import optuna


#### data

In [27]:
# data = open('input.txt', 'r').read()
# data = list(data)

# https://edisciplinas.usp.br/pluginfile.php/3403095/mod_resource/content/1/56ViktorFrankl_Mans%20Search.pdf
text='''
Only slowly could these men be guided back to the commonplace truth that no one has the right to do wrong, not
even if wrong has been done to them. We had to strive to
lead them back to this truth, or the consequences would
have been much worse than the loss of a few thousand stalks
of oats. I can still see the prisoner who rolled up his shirt
sleeves, thrust his right hand under my nose and shouted,
"May this hand be cut off if I don't stain it with blood
on the day when I get home!" I want to emphasize that the
man who said these words was not a bad fellow. He had
been the best of comrades in camp and afterwards.
Apart from the moral deformity resulting from the sudden release of mental pressure, there were two other
fundamental experiences which threatened to damage the
character of the liberated prisoner: bitterness and disillusionment when he returned to his former life.
Bitterness was caused by a number of things he came up
against in his former home town. When, on his return, a
man found that in many places he was met only with a
shrug of the shoulders and with hackneyed phrases, he
tended to become bitter and to ask himself why he had
gone through all that he had. When he heard the same
phrases nearly everywhere—"We did not know about it,"
and "We, too, have suffered," then he asked himself, have
they really nothing better to say to me?
The experience of disillusionment is different. Here it
was not one's fellow man (whose superficiality and lack of
feeling was so disgusting that one finally felt like creeping
into a hole and neither hearing nor seeing human beings
any more) but fate itself which seemed so cruel. A man who
Experiences in a Concentration Camp 99
for years had thought he had reached the absolute limit of
all possible suffering now found that suffering has no limits,
and that he could suffer still more, and still more intensely.
When we spoke about attempts to give a man in camp
mental courage, we said that he had to be shown something
to look forward to in the future. He had to be reminded
that life still waited for him, that a human being waited for
his return. But after liberation? There were some men who
found that no one awaited them. Woe to him who found
that the person whose memory alone had given him courage
in camp did not exist any more! Woe to him who, when the
day of his dreams finally came, found it so different from all
he had longed for! Perhaps he boarded a trolley, traveled
out to the home which he had seen for years in his mind,
and only in his mind, and pressed the bell, just as he has
longed to do in thousands of dreams, only to find that the
person who should open the door was not there, and would
never be there again.
'''

# # make dataset
# text=text.replace('\n','')
# data=list(text)
# # print(data)
# # data=sorted([ord(x) for x in set(data)])
# dataset=[ord(x)-31 for x in data]
# # dataset = map(lambda x:x.strip("8"), lst)
# # [f(x) if condition else g(x) for x in sequence]
# dataset=[0 if x>91 else x for x in dataset] # 0 unk, 1-91
# vocab_size=92
# print(dataset)
# 32space;A65-Z90;a97-z122

from torch.utils.data import Dataset
class Datasetme(Dataset): #https://www.kaggle.com/code/pinocookie/pytorch-dataset-and-dataloader/notebook
    def __init__(self, text, embed_size=4):
        super().__init__()
        text=text.replace('\n','')
        data=list(text)
        self.dataset=[self.encode(x) for x in data] # 0 unk, 1-91
        self.vocab_size=vocab_size=92
        self.embed_size=embed_size
        self.batch_size = 1
        self.embed=nn.Embedding(vocab_size, embed_size)
    def __len__(self):
        return len(self.dataset)-1
    def __getitem__(self, index):
        # return self.dataset[index], self.dataset[index+1]
        return self.get_embed(index), self.get_embed(index+1)
    def embeder(self, char):
        return self.embed(torch.tensor(encode(char)))
    def encode(self, char):
        x = ord(char)-31
        if x>91: x=0
        return x
    def decode(self, x): return chr(x+31)
dataset=Datasetme(text)


#### functions

In [10]:

def off_diagonal(x):
    # print("off_diagonal",x.shape)
    n, m = x.shape
    assert n == m
    return x.flatten()[:-1].view(n - 1, n + 1)[:, 1:].flatten()


#### jepa

In [33]:

class JEPA(nn.Module):
    # def __init__(self, xin_channels, dim_sx, dim_sy, dim_z, dim_v, n_actions, space_dims, hidden_dims):
    def __init__(self, in_dimx, in_dimy, dim_sx, dim_sy, dim_z, dim_v):
    # def __init__(self, vocab_size, dim_sx, dim_sy, dim_z, dim_v):
        super(JEPA, self).__init__()
        self.enc_x = nn.Sequential(
            # nn.Embedding(vocab_size, dim_sx),
            # nn.Flatten(start_dim=1),
            nn.Linear(in_dimx, dim_sx),
            )
        self.enc_y = nn.Sequential(
            # nn.Embedding(vocab_size, dim_sy),
            # nn.Flatten(start_dim=1),
            nn.Linear(in_dimy, dim_sx),
            )
        self.pred = nn.Sequential(
            nn.Linear(dim_sx + dim_z, dim_sy),
            nn.ReLU(True),
            )
        self.exp_x = nn.Sequential(
            nn.Linear(dim_sx, dim_v),
            nn.ReLU(True),
            )
        self.exp_y = nn.Sequential(
            nn.Linear(dim_sy, dim_v),
            nn.ReLU(True),
            )    

    def vicreg(self, x, y): # https://github.com/facebookresearch/vicreg/blob/main/main_vicreg.py
        # invariance loss
        repr_loss = F.mse_loss(x, y)
        x = x - x.mean(dim=0)
        y = y - y.mean(dim=0)

        # variance loss
        std_x = torch.sqrt(x.var(dim=0) + 0.0001)
        std_y = torch.sqrt(y.var(dim=0) + 0.0001)
        std_loss = torch.mean(F.relu(1 - std_x)) / 2 + torch.mean(F.relu(1 - std_y)) / 2

        # # covariance loss
        # cov_x = (x.T @ x) / (self.args.batch_size - 1)
        # cov_y = (y.T @ y) / (self.args.batch_size - 1)
        # cov_loss = off_diagonal(cov_x).pow_(2).sum().div(self.num_features)\
        #  + off_diagonal(cov_y).pow_(2).sum().div(self.num_features)

        # loss = (self.args.sim_coeff * repr_loss + self.args.std_coeff * std_loss + self.args.cov_coeff * cov_loss)

        batch_size=3
        num_features=3
        sim_coeff=1
        std_coeff=1
        cov_coeff=1

        # print("x.dim()",x.dim())
        if x.dim() == 1:
            x = x.view(-1, 1)
        
        if y.dim() == 1:
            y = y.view(-1, 1)
        x=x.T
        y=y.T
        # print("x",x.shape)
        cov_x = (x.T @ x) / (batch_size - 1)
        cov_y = (y.T @ y) / (batch_size - 1)
        # print("cov_x",cov_x.shape)
        cov_loss = off_diagonal(cov_x).pow_(2).sum().div(num_features)\
         + off_diagonal(cov_y).pow_(2).sum().div(num_features)

        loss = (sim_coeff * repr_loss + std_coeff * std_loss + cov_coeff * cov_loss)
        return loss

    # def argm(self, sx, sy):
    def argm(self, SX, SY):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        sampler = optuna.samplers.NSGAIISampler()
        # sampler = optuna.samplers.MOTPESampler()
        pruner = optuna.pruners.MedianPruner()
        batch_size=1
        # if sx.dim() == 2: batch_size,_=sx.shape
        if SX.dim() == 2: batch_size,_=SX.shape
        s=[]
        for i in range(batch_size):
            sx, sy = SX[i],SY[i]
            study = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner)
            # study = optuna.create_study()
            def objective(trial):
                z = trial.suggest_uniform('z', -1, 1)
                # print("z trail",sx,z)
                # z=torch.tensor(z).to(device)
                z=torch.tensor(z).view(1).to(device)
                # print("sx, z",sx.shape, z.shape) #[500, 20] [1]
                sxz = torch.cat([sx, z], dim=-1)
                sy_ = self.pred(sxz)
                mseloss = nn.MSELoss()(sy, sy_)
                return mseloss
            study.optimize(objective, n_trials=10)
            st=study.best_params
            # print("st",st['z'])
            st=torch.tensor([st['z']])
            s.append(st)
        return torch.tensor(s)

    def loss(self, x, y):
        if x.dim()==2: batch_size,_=x.shape
        # print("loss",x,x.shape,x.dtype)
        sx = self.enc_x(x)
        sy = self.enc_y(y)
        sx=sx.flatten(start_dim=1)
        sy=sy.flatten(start_dim=1)
        # print("sx, sy",sx.shape, sy.shape) #10000
        z = self.argm(sx, sy).to(device)
        # z=np.array(z)
        z=torch.tensor(z).view(-1,1)
        # print("sx, z",sx.device, z.device) #[500, 20] [1]
        sxz = torch.cat([sx, z], dim=-1)
        sy_ = self.pred(sxz)
        # loss(sy, sy_)
        mseloss = nn.MSELoss()(sy, sy_)

        vx = self.exp_x(sx)
        vy = self.exp_y(sy)
        # print("vx",vx.shape) #[40]
        vicloss = self.vicreg(vx, vy)
        return mseloss + vicloss

    def forward(self, sx, a):
        # sx = self.enc_x(x)
        # sx=sx.flatten()
        sxz = torch.cat([sx, a], dim=-1)
        # print("sxz",sxz)
        sy_ = self.pred(sxz)
        return sy_
    def encode(self, x): return self.enc_x(x)

vocab_size=dataset.vocab_size
embed_size=dataset.embed_size
# embed_size=4
in_dimx=embed_size # embedding size
in_dimy=embed_size
dim_sx=2
dim_sy=2
dim_z=1
dim_v=5

model = JEPA(in_dimx, in_dimy, dim_sx, dim_sy, dim_z, dim_v).to(device)



#### train eval

In [20]:
from tqdm import tqdm

grad_clip_norm=1
lr=1e-4
betas=(0.9, 0.95)
batch_size=1
def train(loader, model, loss_fn, optimizer):
    model.train()
    losses = []
    pbar = tqdm(enumerate(loader), total = len(loader))
    for it, (x, y) in pbar:
        # print("x,y",x.dtype,y.dtype) #torch.float32 [1,4]
        # print("x,y",x.shape,y.shape)
        x = x.to(device)
        y = y.to(device)
        # print("x",x)
        # with torch.set_grad_enabled(True):

        # logits = model(x)
        # # loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))
        # loss = loss_fn(logits, y)
        loss = model.loss(x,y)
        losses.append(loss.item())

        model.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip_norm)
        optimizer.step()
        # lr = lr
        # pbar.set_description(f"epoch {epoch + 1} iter {it}: train loss {loss.item():.5f}. lr {lr:e}")
        pbar.set_description(f"epoch {epoch + 1} iter {it}: train loss {loss.item():.5f}")


def eval(loader, model, loss_fn):
    model.eval()
    losses = []
    pbar = enumerate(loader)
    for it, (x, y) in pbar:
        x = x.to(device)
        y = y.to(device)
        # with torch.set_grad_enabled(False):
        with torch.no_grad():
            # logits = model(x)
            # # loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))
            # loss = loss_fn(logits, y)
            loss = model.loss(x,y)
            losses.append(loss.item())
    test_loss = float(np.mean(losses))
    logger.info("test loss: %f", test_loss)
    return test_loss


#### wwwwwwwwwww

In [13]:


optimizer = torch.optim.AdamW(model.parameters(), lr = lr, betas = betas)

def loss_fn(logits, y):
    return nn.CrossEntropyLoss()(logits.view(-1, logits.size(-1)), y.view(-1))



In [19]:
from torch.utils.data.dataloader import DataLoader
# train_loader = DataLoader(train_dataset, shuffle = True, pin_memory = True, batch_size = batch_size, num_workers = 4)
# test_loader = DataLoader(test_dataset, shuffle = True, pin_memory = True, batch_size = batch_size, num_workers = 0)
loader = DataLoader(dataset, shuffle = False, batch_size = batch_size, num_workers = 0)

epochs=1
for epoch in range(epochs):
    # run_epoch(train_loader)
    train(loader, model, loss_fn, optimizer)
    test_loss = eval(loader, model, loss_fn)
    print('Test Loss:', test_loss)


epoch 1 iter 5: train loss nan:   0%|          | 2/2663 [00:00<02:28, 17.86it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:113: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])


epoch 1 iter 16: train loss nan:   0%|          | 13/2663 [00:00<01:02, 42.69it/s]

x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:113: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
epoch 1 iter 27: train loss nan:   1%|          | 24/2663 [00:00<00:56, 46.87it/s]

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:113: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
epoch 1 iter 2659: train loss nan: 100%|█████████▉| 2656/2663 [01:00<00:00, 42.91it/s]

x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:113: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
epoch 1 iter 2662: train loss nan: 100%|██████████| 2663/2663 [01:00<00:00, 44.12it/s]

x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])
x,y torch.float32 torch.float32
x,y torch.Size([1, 4]) torch.Size([1, 4])


NameError: ignored

#### inference

In [35]:

context = "This is what "

def encode(char):
    x = ord(char)-31
    if x>91: x=0
    return x
def decode(x): return chr(x+31)

out=[]
for _ in range(20):
    a=torch.zeros(1,1).to(device)
    # x=encode("w")
    x=dataset.embeder("w")
    # print(x)
    x=torch.tensor(x).view(1,-1).to(device)
    sx=model.encode(x)
    # print(sx.shape, a.shape)
    sy_ = model(sx, a)
    out.append(sy_)
print(out)
print(''.join(out))




[tensor([[0.0130, 0.0379]], device='cuda:0', grad_fn=<ReluBackward0>), tensor([[0.0130, 0.0379]], device='cuda:0', grad_fn=<ReluBackward0>), tensor([[0.0130, 0.0379]], device='cuda:0', grad_fn=<ReluBackward0>), tensor([[0.0130, 0.0379]], device='cuda:0', grad_fn=<ReluBackward0>), tensor([[0.0130, 0.0379]], device='cuda:0', grad_fn=<ReluBackward0>), tensor([[0.0130, 0.0379]], device='cuda:0', grad_fn=<ReluBackward0>), tensor([[0.0130, 0.0379]], device='cuda:0', grad_fn=<ReluBackward0>), tensor([[0.0130, 0.0379]], device='cuda:0', grad_fn=<ReluBackward0>), tensor([[0.0130, 0.0379]], device='cuda:0', grad_fn=<ReluBackward0>), tensor([[0.0130, 0.0379]], device='cuda:0', grad_fn=<ReluBackward0>), tensor([[0.0130, 0.0379]], device='cuda:0', grad_fn=<ReluBackward0>), tensor([[0.0130, 0.0379]], device='cuda:0', grad_fn=<ReluBackward0>), tensor([[0.0130, 0.0379]], device='cuda:0', grad_fn=<ReluBackward0>), tensor([[0.0130, 0.0379]], device='cuda:0', grad_fn=<ReluBackward0>), tensor([[0.0130, 0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


TypeError: ignored